In [37]:
# importing some motherfucking libraries
using LinearAlgebra
using Printf

In [194]:
function GCR(A, b, tolerance, max_iter)
    x = zeros(length(b))
    residual = copy(b); # APPARENTLY JULIA LIKES TO BE QUIRKY SO IF B CHANGES THEN SO WILL RESIDUAL OR VICE VERSA SO USE THE COPY()
    res_norm_init = norm(residual);
    
    res_norms = [res_norm_init];
    k = 0;
    p = Float64[];
    Ap = Float64[];
    while (res_norms[k+1]/res_norms[1] > tolerance) && (k <= max_iter)
        k = k+1
        @printf("\nRes norm %lf", res_norms[k]);
        @printf("\nScaled res norm %lf\n", res_norms[k]/res_norms[1]);
        # Use the residual as the first guess for the ne search direction and computer its image
        if k == 1
            p = copy(residual);
            Ap = A * p[:,k];
        elseif k > 1
            p = cat(p, residual, dims=(2,2))
            Ap = cat(Ap, A * p[:,k], dims=(2,2))
        end
        
        #  Make the new Ap vector orthogonal to the previous Ap vectors, and the p vectors A^TA orthogonal to the previous p vectors
        if k > 1
            for j in 1:k-1
               beta    = Ap[:,k]' * Ap[:,j];
               p[:,k]  =  p[:,k] - beta *  p[:,j];
               Ap[:,k] = Ap[:,k] - beta * Ap[:,j];
                
            end
        end
        
        # Make the orthogonal Ap vector of unit length, and scale the p vector so that A * p  is of unit length
        norm_Ap = norm(Ap[:,k]);
        Ap[:,k] = Ap[:,k]/norm_Ap;
        p[:,k] =  p[:,k]/norm_Ap;
        # Determine the optimal amount to change x in the p direction by projecting r onto Ap
        alpha = residual' * Ap[:,k];
        # Update x and r
        x = x + alpha *  p[:,k];

        # Save the norm of r
        res_norms = cat(res_norms, norm(residual), dims=(2,2));
    end

# Did this shit converge or nah
if res_norms[k+1]> (tolerance * res_norms[1])
  @printf("\nBoohoo, the GCR algorithm did NOT converge! Maximum number of iterations reached >:(\n");
  x = [];
else
  @printf("\nSLAY BESTIE! The GCR algorithm converged in %d iterations!\n", k);
end

# Scale the r_norms with respect to the initial residual norm
res_norms = res_norms / res_norms[1];
end

GCR (generic function with 1 method)

In [197]:
# equivalent matlab test commands
# A = [420 0 0 0 0; 0 69 0 0 0; 0 0 420 0 0; 0 0 0 69 0; 0 0 0 0 22]; b = [4; 3; 2; 1; 5]; tgcr(A, b, 0.0001, 300);
# A = [420 2 0 0 0; 0 33 0 0 0; 0 0 62 0 0; 0 0 0 14 0; 0 0 0 0 22]; b = [4; 3; 2; 1; 5]; tgcr(A, b, 0.0001, 300);

#creating test data
N = 5;
A = float([420 0 0 0 0; 0 69 0 0 0; 0 0 420 0 0; 0 0 0 69 0; 0 0 0 0 22]);
b = float([4; 3; 2; 1; 5]);
tol = 0.0001;
iter = 300;

In [198]:
GCR(A, b, tol, iter)


Res norm 7.416198
Scaled res norm 1.000000
alpha = 5.089420893158634
x = [0.010747802916066236, 0.008060852187049676, 0.005373901458033118, 0.002686950729016559, 0.013434753645082793]
residual = [-0.5140772247478189, 2.4438011990935724, -0.25703861237390946, 0.8146003996978575, 4.704435419808179]
Res norm 5.394237
Scaled res norm 0.727359
alpha = 4.609492984826755
x = [0.009483188358214276, 0.0625956796389454, 0.004741594179107138, 0.020865226546315133, 0.1167084430516297]
residual = [0.01706088955000462, -1.3191018950872326, 0.00853044477500231, -0.43970063169574414, 2.4324142528641466]
Res norm 2.801851
Scaled res norm 0.377802
alpha = 2.801851065833676
x = [0.009523809523809523, 0.0434782608695652, 0.0047619047619047615, 0.014492753623188404, 0.22727272727272724]
residual = [1.169203622808368e-15, 4.440892098500626e-16, 5.84601811404184e-16, 1.1102230246251565e-16, 4.440892098500626e-16]
GCR converged in 3 iterations


1×4 Matrix{Float64}:
 1.0  0.727359  0.377802  1.96124e-16